# Implementing Trading with Machine Learning Regression - Part 4

In this notebook, we will use signals and compute trade details and strategy analytics

The key steps for this notebook are:
1. Import the data
2. Analyze the Performance
3. Trade Details and Strategy Analytics
4. Visualise the Entry and Exit
5. Strategy Analysis

In [31]:
## Import the libraries
import numpy as np 
import pandas as pd 

# For Plotting
import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('seaborn-darkgrid')

# To ignore unwanted warnings
import warnings
warnings.filterwarnings("ignore")

### Import Data
We will import the file `regression_strategy_data.csv`. This file has predicted  high, low, signals, gold ETF returns and strategy returns.

In [32]:
strategy_data = pd.read_csv('data/strategy_data.csv', parse_dates=['Date'], index_col='Date')
strategy_data.head()

,Close,High,P_H,Low,P_L,Signal,gld_returns,strategy_returns
Date,,,,,,,,
2017-08-10,122.209999,122.440002,122.581708,122.029999,121.514771,0,NaN,NaN
2017-08-11,122.790001,122.830002,122.747055,122.029999,121.670863,-1,0.004746,0.000000
2017-08-14,121.919998,122.239998,122.504406,121.680000,121.439645,0,-0.007085,0.007085
2017-08-15,120.980003,121.230003,121.153842,120.580002,120.091055,-1,-0.007710,-0.000000
2017-08-16,121.870003,122.059998,121.240462,120.660004,120.164585,-1,0.007357,-0.007357


### Analyze the performannce

First, we will use the Sharpe ratio to analyze the performance. Later in this notebook, we will examine our strategy in detail.

#### Sharpe Ratio
Sharpe ratio is a measure for calculating risk-adjusted return. It is the ratio of the excess return (over risk-free rate) per unit of volatility or standard deviation. The formula for Sharpe Ratio is given by,

`Sharpe ratio = (sqrt(N) x mean(strategy returns - Rf)) / (std dev (Strategy returns - Rf))`

where:
- N is the number of trading days in a year
- Rf is the risk-free rate
- Std dev is standard deviation

In [33]:
# The below function returns the Sharpe ratio for the excess returns
def annualized_sharpe_ratio(returns, N=252):
    return np.sqrt(N) * returns.mean() / returns.std()

# Calculate the excess daily return assuming an annual risk free rate of return of 5%
excess_daily_strategy_return = strategy_data['strategy_returns'] - 0.05/252

# Calculate the annualized sharpe ratio
sharpe = annualized_sharpe_ratio(excess_daily_strategy_return)
print('Sharpe - ', sharpe)

Sharpe -  0.1951213438453711


### Trade Details and Strategy Analytics

Trade analytics takes `data`, `Close`, and `Signal` as input parameters. And it provides us with all information regarding the number of long, short positions, gross profit, and loss using the `trades` dataframe. The synatax of the function is as follows:

    get_trades()

Parameters:
1. data: dataframe to be used for trade analysis
2. Close: Close price column of dataframe
3. Signal: Signal column of dataframe

Returns:
1. Position - long or short
2. Entry date of long or short position
3. Entry price of long or short position
4. Exit Data of long or short position
5. Exit price of long or short position

In [34]:
def get_trades(data, close_column, signal_column):
    """Function to generate trade details
    """
    # Intializing trades dataframe
    trades = pd.DataFrame()

    # There are no current positions open at the beginning
    current_position = 0

    # Creating an empty string for entry_date
    entry_date = ''

    for i in data.index:

        # Save the signal generated on date i in 'new_position'
        new_position = data.loc[i, signal_column]

        # We don't take any action if the current_position is equal to the signal generated at date i.
        # The below code will run if the signal generated is not equal to current_position.
        if new_position != current_position:

            # The below code will run if the entry_date is not empty
            if entry_date != '':

                # save entry price in entry_price
                entry_price = data.loc[entry_date, close_column]

                # Since the signal and current_position are not equal, we exit the position at date i
                exit_date = i

                # Save exit price in exit_price
                exit_price = data.loc[exit_date, close_column]

                # Create a dataframe trade_details with trade details
                trade_details = pd.DataFrame(
                    [(current_position, entry_date, entry_price, exit_date, exit_price)])

                # Update the trades dataframe in every loop
                trades = trades.append(trade_details, ignore_index=True)

                # Reset the entry_date
                entry_date = ''

            # If the new_position i.e. signal generated at i is not '0' and also not equal to current_position, we will enter either short or long trade
            if new_position != 0:

                # Save entry date of trade in entry_date
                entry_date = i

            # Update the current_position with signal generated on date i
            current_position = new_position

    # Creating below five columns in trades dataframe.
    trades.columns = ['Position', 'Entry Date',
                      'Entry Price', 'Exit Date', 'Exit Price']

    # Create a column in trades dataframe to store the profit and loss from each trade
    trades['PnL'] = (trades['Exit Price'] -
                     trades['Entry Price']) * trades['Position']

    # Returns 'trades' dataframe
    return trades

In [36]:
# Using get_trades function to evaluate trades detail
trades = get_trades(strategy_data, 'Close', 'Signal')
print('Trade Details')

# Printing trades dataframe
trades.head()

Trade Details


,Position,Entry Date,Entry Price,Exit Date,Exit Price,PnL
0,-1,2017-08-11,122.790001,2017-08-14,121.919998,0.870003
1,-1,2017-08-15,120.980003,2017-08-17,122.489998,-1.509995
2,1,2017-08-17,122.489998,2017-08-21,122.760002,0.270004
3,1,2017-08-22,122.209999,2017-08-23,122.669998,0.459999
4,1,2017-08-25,122.739998,2017-08-28,124.690002,1.950004
